<a href="https://colab.research.google.com/github/maz-joko78/VTCI-LANDSAT-LC08-C02-T1_L2/blob/main/Harmonic_Mean_Sound_Speed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 📌 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📌 2. Baca file dari Google Drive
import pandas as pd
import numpy as np

# Ganti path sesuai lokasi file Anda
file_path = "/content/drive/MyDrive/Data_ADHO/Data_ADHO_SVP.txt"

# Asumsikan file berformat: index depth velocity (tanpa header)
df = pd.read_csv(file_path, delim_whitespace=True, names=["Index", "Depth", "Velocity"])

# 📌 3. CONSTANT VELOCITY METHOD
df['dz_cv'] = df['Depth'].diff().fillna(0)
df['t_cv'] = df['dz_cv'] / df['Velocity']
CH_cv = df['dz_cv'].sum() / df['t_cv'].sum()
STD_cv = df['Velocity'].std()

# 📌 4. CONSTANT GRADIENT METHOD
df['C_next'] = df['Velocity'].shift(-1)
df['Depth_next'] = df['Depth'].shift(-1)
df['dz_cg'] = df['Depth_next'] - df['Depth']
df['g'] = (df['C_next'] - df['Velocity']) / df['dz_cg']

# Hitung t_i dengan kondisi nilai gradient ≠ 0 dan nilai kecepatan valid
df['t_cg'] = np.where(
    (df['g'] != 0) & (df['Velocity'] > 0) & (df['C_next'] > 0),
    (1 / df['g']) * np.log(df['C_next'] / df['Velocity']),
    0
)

# Ambil hanya nilai t_cg yang valid (>0)
valid_cg = df[df['t_cg'] > 0]
CH_cg = valid_cg['dz_cg'].sum() / valid_cg['t_cg'].sum()
STD_cg = valid_cg['Velocity'].std()

# 📌 5. Cetak hasil
print("📊 Hasil Perhitungan Harmonic Mean Sound Speed (CH):")
print(f"- Metode Constant Velocity : {CH_cv:.4f} m/s")
print(f"- Metode Constant Gradient : {CH_cg:.4f} m/s")
print("\n📉 Standard Deviasi (STDDEV) Kecepatan:")
print(f"- Metode Constant Velocity : {STD_cv:.4f} m/s")
print(f"- Metode Constant Gradient : {STD_cg:.4f} m/s")


Mounted at /content/drive


/tmp/ipython-input-2-615488347.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, names=["Index", "Depth", "Velocity"])


📊 Hasil Perhitungan Harmonic Mean Sound Speed (CH):
- Metode Constant Velocity : 1577.6242 m/s
- Metode Constant Gradient : 1561.9876 m/s

📉 Standard Deviasi (STDDEV) Kecepatan:
- Metode Constant Velocity : 24.8932 m/s
- Metode Constant Gradient : 22.0012 m/s
